# flask_socketio

WebSocket communication protocols facilitate the real-time transfer of data between the client and the server. 

In [ ]:
from flask import Flask, render_template
from flask_socketio import SocketIO, emit

# Initialize the flask web app
app = Flask(__name__)
app.config['SECRET_KEY'] = 'your-secret-key'
socketio = SocketIO(app, cors_allowed_origins="*")  # Optional: allow CORS  It prevents your frontend from accidentally leaking data to or accepting data from unknown sources.

@app.route('/')
def index():
    return render_template('index.html')

# This allows us to have a custom socketio event listener: 'chat_message'
@socketio.on('chat_message')
def handle_message(msg):
    print('Chat message received: ' + msg)
    # emit allows us to send this response to a specific socketio event on the frontend
    emit('chat_response', msg, broadcast=True)  # Custom event sent to frontend

if __name__ == '__main__':
    socketio.run(app, debug=True, allow_unsafe_werkzeug=True, port=8000, host='0.0.0.0')

### Default socket io event listener

```@socketio.on('message')``` \
```def handle_message(msg):``` \
```    print("Message received:", msg)```

## Frontend

<hr>

**step 1: Include the Socket.IO client library** \
```<script src="https://cdn.socket.io/4.7.2/socket.io.min.js"></script>```
- This loads the Socket.IO JavaScript client from the official CDN.
- It’s necessary for enabling real-time WebSocket communication with the Flask-SocketIO server.

<hr>

**step 2: Connect to the server** \
```const socket = io();```
- This line automatically attempts to connect to the server that served the page.
- If your server is running on the same origin (http://localhost:8000), this works automatically.
- If your server is on a different host or port, you must do:
```const socket = io("http://localhost:8000");```

<hr>

**step 3: Listen for messages from the server** \
```socket.on('chat_response', function(msg) {```\
```    const li = document.createElement('li');```\
```    li.textContent = msg;```\
```    document.getElementById('messages').appendChild(li);```\
```});``` 
- This sets up a listener for an event called 'chat_response'.
- Whenever the server emits a 'chat_response' message, this function is triggered.
- It adds the received message as a new li item in the ul with ID messages.
- **<div style="color: red;"> msg is passed the value sent from the backend</div>**

🔁 This is what makes the app real-time: when the server broadcasts a message, all connected clients update instantly.

<hr>

**step 4: Send a message to the server** \
```function sendMessage() {```\
```    const input = document.getElementById('message');```\
```    const message = input.value;```\
```    socket.emit('chat_message', message);  // Emit to server```\
```    input.value = '';```\
```}```
- This function gets triggered when the user clicks the Send button.
- It reads the message from the input field.
- Then it emits an event called 'chat_message' and sends it to the server.
- The server must have a listener for 'chat_message' (usually in your Flask backend), like:

```@socketio.on('chat_message')```\
```def handle_message(msg):```\
```    emit('chat_response', msg, broadcast=True)```

🔁 That’s how the message goes from client → server → all clients.

In [ ]:
<!DOCTYPE html>
<html>
<head>
    <title>Chat</title>
    <script src="https://cdn.socket.io/4.7.2/socket.io.min.js"></script>
</head>
<body>
    <h2>Chat Room</h2>
    <ul id="messages"></ul>
    <input id="message" type="text" placeholder="Type your message here" />
    <button onclick="sendMessage()">Send</button>

    <script>
        const socket = io();

        // Receive messages from the 'chat_response' event
        socket.on('chat_response', function(msg) {
            const li = document.createElement('li');
            li.textContent = msg;
            document.getElementById('messages').appendChild(li);
        });

        function sendMessage() {
            const input = document.getElementById('message');
            const message = input.value;
            socket.emit('chat_message', message);  // Emit custom event to backend
            input.value = '';
        }
    </script>
</body>
</html>